In [ ]:
# This is a wrapper for a command line operation using Robospect and Ken Carrell's 
# normalization script to

# A. REPRODUCE our steps for finding a, b, c, d
# B. APPLY our solution to any other spectra to find [Fe/H]

# written by E.S.

In [4]:
import subprocess
from subprocess import call
from subprocess import Popen
import shlex
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os, os.path
from os import listdir
from os.path import isfile, join
import pandas as pd
import sys
from pylab import * 
import glob
from IPython.display import clear_output
from astropy.io import fits

In [30]:
class norm_spec:
    
    ##############################################################################
    # STEP 1: NORMALIZE SPECTRA (applicable to A and B)
    ##############################################################################
    
    # this is a superclass that will be inherited by other classes
    
    def __init__(self, input_file):
        self.smoothing = 22 # smoothing applied by Carrell's normalization code
        self.input_file = input_file
        
    def __call__(self):
        
        # compile Carrell's normalization code
        # (see carrell_readme.txt)
        
        # Carrell's C code should already be compiled by setup.py. But here are the manual commands for posterity:
        #normzn_compile1 = shlex.split("g++ -o bkgrnd bkgrnd.cc")
        #normzn_compile2 = subprocess.Popen(normzn_compile1) # run
        
        # run the normalization routine on the data
        normzn_run1 = shlex.split("./bkgrnd --smooth "+str(self.smoothing)+" "+self.input_file) # self.input_file can be 'in.data'
        normzn_run2 = subprocess.Popen(normzn_run1, stdout=subprocess.PIPE, stderr=subprocess.PIPE) # run and capture output
                                  
        # make list of all output files
        dir_name = "test_output/"
        list_output_files = [name for name in os.listdir(dir_name) if os.path.isfile(os.path.join(dir_name, name))]
                                  
        # divide the second column of the output files (empirical) with the third (normalization) 
        header = ['WAVELENGTH', 'FLUX'] # headers of output file that Robospect will use (see Robospect user manual)
        for filenum in range(0,len(list_output_files)):
            df = pd.read_csv('test_output/'+str(list_output_files[filenum]), delim_whitespace=True, header=None)
            df['WAVELENGTH'] = df[0]
            df['FLUX'] = np.divide(df[1],df[2]) # normalize
            df.to_csv('test_normzed_output/output.csv', columns = header, index = False, sep = ' ') # write out file 
            del df                          
                                  

In [31]:
# example: run normalization of raw data

do_normzn = norm_spec("input_file") # initialize class instance
# do_normzn.smoothing = 22 # can overload default smoothing
do_normzn() # call it

In [ ]:
#########################################################################################################################

In [ ]:
class generate_synthetic_spec(norm_spec):
    
    ##############################################################################
    # STEP 2: TAKE EMPIRICAL SPECTRA AND CHANGE NOISE CHARACTERISTICS TO GENERATE SYNTHETIC SPECTRA
    # THIS WILL INVOLVE RUNNING RW'S FORTRAN SCRIPT (applicable to A)
    ##############################################################################
    
    # inherits norm_spec class for normalizing spectra
    
    # normalize the synthetic spectra

In [59]:
#subprocess.call(["gfortran","-o","hello","junk.f90"]) # create
#subprocess.call(["./hello"])   # subprocess.check_output(["./hello"]) lets you see the output

0

In [ ]:
#########################################################################################################################

In [ ]:
def run_robospect():
### THIS NOT WORKING!!!

    ##############################################################################
    # STEP 3: RUN ROBOSPECT ON ANY SPECTRA AND WRITE OUT EW VALUES AS *.c.dat FILES (applicable to A and B)
    ##############################################################################
    
    # for applying to synthetic spectra 

    # accumulate list of filenames of normalized synthetic spectra
    fileNameList = glob.glob("../*_*.c.dat") # (or search whatever other directory the *.c.dat files are in)

    # for-loop to write out *.robolines and *.robospect files
    for p in fileNameList: 
        # default command: (-F: find all lines; -P: sets path of output files)
        # robospect -F -P rs.out example.dat
        args = ['./src/robospect', '-F', '-P', 'rs.out', p]
        q = subprocess.call(args, shell=True)

In [ ]:
# N.b. Wrapper should catch anything that robospect tries to print to terminal 
# (3 pipes in any process: StdIn, StdOut, and StdError)

In [ ]:
#########################################################################################################################

In [ ]:
class scraper():

    ##############################################################################
    # STEP 3B: SCRAPE ALL THE EW INFO FROM *.c.dat FILES (applicable to A and B)
    ##############################################################################
    
    def __init__(self):
        
        # directory containing the directory containing *.c.dat files
        self.stem = '/home/../../media/unasemaje/Seagate Expansion Drive/rrlyrae_data_reduction/'
        # subdirectory containing the *.c.dat files
        self.subdir = 'McDrealiz'
        
        # get list of filenames without the path
        fileListLong = glob.glob(stem+subdir+'/'+'*.fits.robolines')
        fileListUnsorted = [os.path.basename(x) for x in fileListLong]
        fileList = sorted(fileListUnsorted)
        
    def __call__(self):
        
        # sanity check: are the lines listed in order?
        def line_check(lineCenters):
            if ((lineCenters[0] < 3933.660-10) or (lineCenters[0] > 3933.660+10)): # CaIIK
            print('Lines not matching!')
            sys.exit  # ... and abort
        elif ((lineCenters[1] < 3970.075-10) or (lineCenters[1] > 3970.075+10)): # H-epsilon (close to CaIIH)
            print('Lines not matching!')
            sys.exit
        elif ((lineCenters[2] < 4101.7100-10) or (lineCenters[2] > 4101.7100+10)): # H-delta
            print('Lines not matching!')
            sys.exit
        elif ((lineCenters[3] < 4340.472-10) or (lineCenters[3] > 4340.472+10)): # H-gamma
            print('Lines not matching!')
            sys.exit
        elif ((lineCenters[4] < 4861.290-10) or (lineCenters[4] > 4861.290+10)): # H-beta
            print('Lines not matching!')
            sys.exit
        return
    
    
        # loop over all filenames, extract line data
        for t in range(0,len(fileList)):
        
            # read in Robospect output
            df = pd.read_csv(stem+subdir+'/'+fileList[t], header=13, delim_whitespace=True, index_col=False, usecols=np.arange(17))
    
            # check lines are in the right order
            line_check(df['#x0'])
    
            # add two cols on the left: the filename, and the name of the line
            sLength = len(df['mean']) # number of lines (should be 5)
            df['file_name'] = pd.Series(fileList[t], index=df.index)
            df['synth_spec_name'] = pd.Series(fileList[t].split(".")[0], index=df.index) # multiple synthetic spectra correspond to one empirical spectrum
            df['empir_spec_name'] = pd.Series(fileList[t].split(".")[0][0:-4], index=df.index) # empirical spectrum
            #df['star_name'] = pd.Series(fileList[t].split("__")[0], index=df.index)
            df['line_name'] = ['CaIIK', 'Heps', 'Hdel', 'Hgam', 'Hbet']
    
            # get an idea of the progress
            clear_output(wait=True)
            print('Out of '+str(len(fileList))+' files, '+str(t)+' scraped...')
    
            # if this is the first list, start a master copy from it to concatenate stuff to it
            if (t==0):
                dfMaster = df.copy()
            else:
                dfMaster = pd.concat([dfMaster,df])
                del df # clear variable
                
        # write to csv, while resetting the indices
        # note THIS TABLE INCLUDES ALL DATA, GOOD AND BAD
        dfMaster_reset = dfMaster.reset_index(drop=True).copy() # this gets shown further down in this notebook
        dfMaster.reset_index(drop=True).to_csv(stem+subdir+'/'+subdir+'_largeTable_test.csv') # this is effectively the same, but gets written out
        
        ## IF WE ARE INTERESTED IN SPECTRA THAT HAVE ALL WELL-FIT LINES
        # identify the synthetic spectrum names which have at least one line with a bad fit
        badSynthSpectra = dfMaster_reset['synth_spec_name'][np.squeeze(whereRedFlag)]
        # remove duplicate names
        badSynthSpectra_uniq = badSynthSpectra.drop_duplicates()
        # keep only the spectra that have all lines well-fit
        dfMaster_reset_dropBadSpectra = dfMaster_reset.where(~dfMaster_reset['synth_spec_name'].isin(badSynthSpectra_uniq))
        
        # write to csv
        # note THIS TABLE HAS SPECTRA WITH ANY BAD ROWS REMOVED
        dfMaster_reset_dropBadSpectra.to_csv(subdir+'_largeTable_bad_spectra_removed_test.csv') # this is effectively the same, but gets written out

In [31]:
# example: run the scraper 

do_scrape = scraper() # initialize class instance
do_scrape() # call it

In [ ]:
#########################################################################################################################

In [ ]:
class findHK():
    
    ##############################################################################
    # STEP 4: READ IN ROBOSPECT EWS OF SYNTHETIC SPECTRA, RESCALE THEM, AVERAGE THEM, PLOT H-K SPACE (applicable to A and B)
    ##############################################################################
    
    def __init__(self):
    
        # read in line data
        line_data = pd.read_csv('McDrealiz_largeTable_2017jan21_bad_spectra_removed.csv', delim_whitespace=False)
        
        # initialize arrays: essential info
        empir_spec_name_array = []
        star_name_array = []
        H_data_array = []
        K_data_array = []
        err_H_data_array = [] 
        err_K_data_array = []

        # initialize arrays: other info
        Hbet_data_array = []
        err_Hbet_data_array = []
        Hgam_data_array = []
        err_Hgam_data_array = []
        rHgam_data_array = [] # rescaled Hgamma
        err_rHgam_data_array = []
        Hdel_data_array = []
        err_Hdel_data_array = []
        Heps_data_array = []
        err_Heps_data_array = []
        
    def __call__(self):
        
        # make a list of all UNIQUE, EMPIRICAL spectrum names
        uniqueSpecNames = line_data.drop_duplicates(subset='empir_spec_name')['empir_spec_name']
        
        # fit a straight line to Hgam vs Hdel
        x_data = line_data['EQW'].where(line_data['line_name'] == 'Hdel').dropna() # Hdel
        y_data = line_data['EQW'].where(line_data['line_name'] == 'Hgam').dropna() # Hgam
        Hgam = np.copy(y_data)
        m,b = np.polyfit(x_data, y_data, 1) # might want errors later, too 
        
        # generate a rescaled Hgam, call it rHgam
        rHgam_all = np.divide(np.subtract(Hgam,b),m)
        
        # prepare data for a plot
        # loop over every EMPIRICAL spectrum and assemble SYNTHETIC data into arrays
        for p in range(0,len(uniqueSpecNames)):
    
            # the name of the empirical spectrum being used here
            print(np.array(uniqueSpecNames)[p])
    
            # extract all synthetic data corresponding to this empirical spectrum
            data_for_this_empir_spectrum = line_data.where(line_data['empir_spec_name'][0:-4] == np.array(uniqueSpecNames)[p])
    
            # scrape data
            raw_Hbet_data = data_for_this_empir_spectrum['EQW'].where(line_data['line_name'] == 'Hbet')
            raw_Hgam_data = data_for_this_empir_spectrum['EQW'].where(line_data['line_name'] == 'Hgam')
            raw_Hdel_data = data_for_this_empir_spectrum['EQW'].where(line_data['line_name'] == 'Hdel')
            raw_Heps_data = data_for_this_empir_spectrum['EQW'].where(line_data['line_name'] == 'Heps')
            raw_K_data = data_for_this_empir_spectrum['EQW'].where(line_data['line_name'] == 'CaIIK')
    
            # rescale and remove nans
            Hbet_data_wnans = np.array(np.copy(raw_Hbet_data))
            Hgam_data_wnans = np.array(np.copy(raw_Hgam_data))
            Hdel_data_wnans = np.array(np.copy(raw_Hdel_data))
            Heps_data_wnans = np.array(np.copy(raw_Heps_data))    
            K_data_wnans = np.array(np.copy(raw_K_data))
            rHgam_data_wnans = np.array(np.divide(np.subtract(raw_Hgam_data,b),m)) # rescale Hgam EWs
    
            Hbet_data = Hbet_data_wnans[np.isfinite(Hbet_data_wnans)] # remove nans
            Hgam_data = Hgam_data_wnans[np.isfinite(Hgam_data_wnans)]
            Hdel_data = Hdel_data_wnans[np.isfinite(Hdel_data_wnans)]
            Heps_data = Heps_data_wnans[np.isfinite(Heps_data_wnans)]
            rHgam_data = rHgam_data_wnans[np.isfinite(rHgam_data_wnans)]
            K_data = K_data_wnans[np.isfinite(K_data_wnans)]
    
            # get the H-K synthetic data together
            balmer_data_allsynthetic_spec = np.mean([Hdel_data,rHgam_data], axis=0) # Balmer EW = 0.5*(Hdel + rHgam)
            K_data_allsynthetic_spec = np.copy(K_data)
    
            # the actual points to plot (or record in a table)
            Hbet_data_pt = np.nanmedian(Hbet_data)
            Hgam_data_pt = np.nanmedian(Hgam_data)
            rHgam_data_pt = np.nanmedian(rHgam_data)
            Hdel_data_pt = np.nanmedian(Hdel_data)
            Heps_data_pt = np.nanmedian(Heps_data)
            balmer_data_pt = np.nanmedian(balmer_data_allsynthetic_spec)
            K_data_pt = np.nanmedian(K_data_allsynthetic_spec)
    
            # the error bars
            err_Hbet_data = np.nanstd(Hbet_data)
            err_Hgam_data = np.nanstd(Hgam_data)
            err_rHgam_data = np.nanstd(rHgam_data)
            err_Hdel_data = np.nanstd(Hdel_data)
            err_Heps_data = np.nanstd(Heps_data)
            err_balmer_data = np.nanstd(balmer_data_allsynthetic_spec)
            err_K_data = np.nanstd(K_data_allsynthetic_spec)
    
            #plt.plot(balmer_data_pt,K_data_pt)
            #plt.errorbar(balmer_data_pt, K_data_pt, yerr=err_K_data, xerr=err_balmer_data)

            # append data to arrays: essential info
            empir_spec_name_array = np.append(empir_spec_name_array,np.array(uniqueSpecNames)[p])
            star_name_array = np.append(star_name_array,str(np.array(uniqueSpecNames)[p])[0:-3])
            H_data_array = np.append(H_data_array,balmer_data_pt)
            err_H_data_array = np.append(err_H_data_array,err_balmer_data)
            K_data_array = np.append(K_data_array,K_data_pt)
            err_K_data_array = np.append(err_K_data_array,err_K_data)
    
            # append data to arrays: other info
            Hbet_data_array = np.append(Hbet_data_array,Hbet_data_pt)
            err_Hbet_data_array = np.append(err_Hbet_data_array,err_Hbet_data)
            Hgam_data_array = np.append(Hgam_data_array,Hgam_data_pt)
            err_Hgam_data_array = np.append(err_Hgam_data_array,err_Hgam_data)
            rHgam_data_array = np.append(rHgam_data_array,err_rHgam_data) # rescaled Hgamma
            err_rHgam_data_array = np.append(err_rHgam_data_array,err_rHgam_data)
            Hdel_data_array = np.append(Hdel_data_array,Hdel_data_pt)
            err_Hdel_data_array = np.append(err_Hdel_data_array,err_Hdel_data)
            Heps_data_array = np.append(Heps_data_array,Heps_data_pt)
            err_Heps_data_array = np.append(err_Heps_data_array,err_Heps_data)
    
            # clear some variables
            balmer_data_allsynthetic_spec=None 
            K_data_allsynthetic_spec=None
            balmer_data_allsynthetic_spec=None 
            K_data_allsynthetic_spec=None
            
        # put everything into a dataframe

        d = {'empir_spec_name': empir_spec_name_array, 
             'star_name': star_name_array,
             'Hbet': Hbet_data_array,
             'err_Hbet': err_Hbet_data_array,
             'Hgam': Hgam_data_array,
             'err_Hgam': err_Hgam_data_array,
             'Hdel': Hdel_data_array,
             'err_Hdel': err_Hdel_data_array,
             'Heps': Heps_data_array,
             'err_Heps': err_Heps_data_array, 
             'rHgam': rHgam_data_array,
             'err_rHgam': err_rHgam_data_array,  
             'balmer': H_data_array,
             'err_balmer': err_H_data_array,
             'K': K_data_array,
             'err_K': err_K_data_array
            }     
        df_collation = pd.DataFrame(data=d)
        
        # read in a text file containing phase information
        phase_info = pd.read_csv("~/Documents/PythonPrograms/all_Python_code/2016_08_27_rrlyrae_metal_fit_emcee_wrapper/eckhart_2ndPass_allSNR_noVXHer_lowAmpPrior.csv")
        
        # paste phase info into the table of EWs
        phase_array = []
        feh_array = []
        err_feh_array = []
        name_array = []

        for q in range(0,len(df_collation)):
            name_this_one = phase_info['Spectrum'].where(phase_info['Spectrum'] == df_collation['empir_spec_name'][q]).dropna()
            phase_this_one = phase_info['phase'].where(phase_info['Spectrum'] == df_collation['empir_spec_name'][q]).dropna()
            feh_this_one = phase_info['FeH'].where(phase_info['Spectrum'] == df_collation['empir_spec_name'][q]).dropna()
            err_feh_this_one = phase_info['eFeH'].where(phase_info['Spectrum'] == df_collation['empir_spec_name'][q]).dropna()
            name_array = np.append(name_array,name_this_one)
            phase_array = np.append(phase_array,phase_this_one)
            feh_array = np.append(feh_array,feh_this_one)
            err_feh_array = np.append(err_feh_array,err_feh_this_one)
        df_collation_real = df_collation.dropna().copy(deep=True) # drop row of nans
        df_collation_real['phase'] = phase_array
        df_collation_real['FeH'] = feh_array
        df_collation_real['eFeH'] = err_feh_array
        
        # write to csv
        df_collation_real.to_csv('more_realistic_EWs_w_phase_test.csv')
        
        # make plot: each color is a different star, open circles are bad phase region
        data_to_plot = pd.read_csv('more_realistic_EWs_w_phase.csv') # read data back in
        
        # make list of unique star names 
        unique_star_names = data_to_plot.drop_duplicates(subset=['star_name'])['star_name'].values
        
        # plot data points
        cmap = plt.get_cmap(name='jet')
        fig = plt.figure()
        ax = fig.add_subplot(111)

        # loop over every star, overlay the set of points for that star on the plot
        for y in range(0,len(unique_star_names)):
    
            x_data = data_to_plot['balmer'].where(data_to_plot['star_name'] == unique_star_names[y])
            y_data = data_to_plot['K'].where(data_to_plot['star_name'] == unique_star_names[y])
    
            err_x_data = data_to_plot['err_balmer'].where(data_to_plot['star_name'] == unique_star_names[y])
            err_y_data = data_to_plot['err_K'].where(data_to_plot['star_name'] == unique_star_names[y])
    
            # plot, and keep the same color for each star
            color_this_star = cmap(float(y)/len(unique_star_names))
            ax.errorbar(x_data,y_data,yerr=err_y_data,xerr=err_x_data,linestyle='',fmt='o',markerfacecolor=color_this_star,color = color_this_star)
    
            x_data_badPhase = x_data.where(np.logical_or(data_to_plot['phase'] > 0.8, data_to_plot['phase'] < 0.05))
            y_data_badPhase = y_data.where(np.logical_or(data_to_plot['phase'] > 0.8, data_to_plot['phase'] < 0.05))
    
            # overplot unfilled markers to denote bad phase region
            ax.errorbar(x_data_badPhase,y_data_badPhase,linestyle='',fmt='o',markerfacecolor='white',color = color_this_star)
    
            # add star name
            ax.annotate(unique_star_names[y], xy=(np.array(x_data.dropna())[0], 
                                          np.array(y_data.dropna())[0]), 
                xytext=(np.array(x_data.dropna())[0], np.array(y_data.dropna())[0]))
    
        plt.title('KH plot, using synthetic spectra')
        plt.ylabel('CaIIK EW (milliangstrom)')
        plt.xlabel('Balmer EW (milliangstrom)')
        plt.show()

In [31]:
# example: run the findHK

do_HK = findHK() # initialize class instance
do_HK() # call it

In [ ]:
class lit_metallicities():
    
    ##############################################################################
    # STEP X: READ IN LITERATURE METALLICITY VALUES AND RESCALE
    ##############################################################################
    
    def __init__(self):
    
        stem = "~/Documents/PythonPrograms/all_Python_code/2018_03_31_rrlyrae_rescale_a_la_chadid/"
        
        # Fe/H from Layden+ 1994
        layden_feh = pd.read_csv(stem + "layden_1994_abundances.dat",delimiter=';')
        # RES: "rather low"
        
        # Fe/H Clementini+ 1995
        clementini_feh = pd.read_csv(stem + "clementini_1995_abundances.dat")

        # Fe/H Fernley+ 1996
        fernley_feh = pd.read_csv(stem + "fernley_1996_abundances.dat")
        # RES: 60,000, FeI & FeII, 5900-8100 A
        
        # log(eps) from Lambert+ 1996
        lambert_logeps = pd.read_csv(stem + "lambert_1996_abundances.dat")
        # RES: ~23,000, FeII + photometric models, 3600-9000 A
        
        # Fe/H from Wallerstein and Huang 2010, arXiv 1004.2017
        wallerstein_feh = pd.read_csv(stem + "wallerstein_huang_2010_abundances.dat")
        # RES: ~30,000, FeII
        
        # Fe/H from Chadid+ 2017 (FeI and II lines)
        chadid_feh = pd.read_csv(stem + "chadid_2017_abundances.dat")
        # RES: 38000, FeI & FeII, 3400-9900 A

        # Fe/H from Liu+ 2013
        liu_feh = pd.read_csv(stem + "liu_2013_abundances.dat")
        # RES: ~60,000, FeI (& FeII?), 5100-6400 A

        # Fe/H from Nemec+ 2013
        nemec_feh = pd.read_csv(stem + "nemec_2013_abundances.dat")
        # RES: ~65,000 or 36,000, FeI & FeII, 5150-5200 A

        # Fe/H from Fernley+ 1997
        fernley97_feh = pd.read_csv(stem + "fernley_1997_abundances.dat",delimiter=';')
        # RES: 60,000, two FeII lines, 5900-8100 A

        # Fe/H from Solano+ 1997
        solano_feh = pd.read_csv(stem + "solano_1997_abundances.dat",delimiter=';')
        # RES: 22,000 & 19,000, strong FeI lines, 4160-4390 & 4070-4490 A
        
        # Fe/H from Pacino+ 2015
        pacino_feh = pd.read_csv(stem + "pacino_2015_abundances.dat") 
        # RES: >30,000, FeI (weighted average), 4000-8500 A

        # Fe/H from Sneden+ 2017
        sneden_feh = pd.read_csv(stem + "sneden_2017_abundances.dat")
        # RES: ~27,000 (at 5000 A), FeI & FeII, 3400-9000 A
        
        # convert Lambert's values, which are in terms of log(eps)
        # FeH = log(epsFe) - log(epsFe,sol)
        #     = log(epsFe) - log(NFe,sol/NH,sol)
        #     = log(epsFe) - 7.51 # value of 7.51 from Anstee+ 1997, MNRAS
        lambert_logeps['feh'] = np.subtract(lambert_logeps['log_eps_fe_spec'], 7.51) 
        
        # average the values in Chadid from FeI and FeII lines
        chadid_feh['feh'] = np.mean([chadid_feh[' fehI'].values,chadid_feh[' fehII'].values],axis=0)
        
        ## ## INCLUDE SINGLE DATA PT FROM KOLENBERG+ 2010? (SEE CHADID+ 2017, FIG. 7)
        
        # FYI: average Fe/H values in Liu+ 2013 which were taken at different phases
        # liu_feh.groupby(liu_feh['name'], axis=0, as_index=False).mean()
        
        # FYI: average Fe/H values in Sneden+ 1997 which were taken at different epochs
        # sneden_feh.groupby(sneden_feh['name'], axis=0, as_index=False).mean()
        
        # initialize arrays: essential info
        empir_spec_name_array = []
        star_name_array = []
        H_data_array = []
        K_data_array = []
        err_H_data_array = [] 
        err_K_data_array = []

        # initialize arrays: other info
        Hbet_data_array = []
        err_Hbet_data_array = []
        Hgam_data_array = []
        err_Hgam_data_array = []
        rHgam_data_array = [] # rescaled Hgamma
        err_rHgam_data_array = []
        Hdel_data_array = []
        err_Hdel_data_array = []
        Heps_data_array = []
        err_Heps_data_array = []
        
    def __call__(self):
        
        # make a list of all UNIQUE, EMPIRICAL spectrum names
        uniqueSpecNames = line_data.drop_duplicates(subset='empir_spec_name')['empir_spec_name']
        
        
    # fcn: find stars that overlap with Layden 1994, and return (x,y,z)=(FeH_Lay94,FeH_input-FeH_Lay94,starname)
    def find_match_Layden(input_table, layden_table, plot_name, offset=False):
    
        inputFeH = []
        laydenFeH = []
        nameArray = []
    
        for row in range(0,len(input_table)): # scan over each row in input table
            if (layden_table['name'] == input_table['name'][row]).any():
            #if (input_table['name'][row] isin layden_table['name']): # if there's a star name that matches
                inputFeH = np.append(inputFeH,input_table['feh'][row])
                laydenFeH = np.append(laydenFeH,layden_table.loc[layden_table['name'] == input_table['name'][row]]['feh'])
                nameArray = np.append(nameArray,input_table['name'][row])
    
        residuals = np.subtract(inputFeH,laydenFeH)
    
        # best-fit line
        coeff = np.polyfit(laydenFeH, residuals, 1)
        limits = [-3.0,0.5]
        line = np.multiply(coeff[0],limits)+coeff[1]
    
        # if there needs to be an offset (like in Fig. 6 of Chadid+ 2017)
        chadid_y_125 = -0.10583621694962 # from Chadid line at Fe/H=-1.25
        this_y_125 = np.multiply(coeff[0],-1.25)+coeff[1] # y-value of this line at Fe/H=-1.25
        net_offset = chadid_y_125 - this_y_125 # offset needed to move line
        print('Y_offset to overlap with Chadid+ 2017 at Fe/H=-1.25:')
        print(net_offset)
        print('Number of overlapping stars:')
        print(len(residuals))
        line_offset = np.add(line,net_offset)
    
        # save a plot
        plt.scatter(laydenFeH, np.subtract(inputFeH,laydenFeH))
        plt.plot([-3.0,0.5], [0., 0.], linestyle='--')
        plt.plot(limits, line)
        plt.plot(limits, line_offset)
        #plt.xlim([-3.0,0.5])
        #plt.ylim([-0.6,0.6])
        plt.xlabel('[Fe/H]_Lay94')
        plt.ylabel('[Fe/H]_input - [Fe/H]_Lay94')
        plt.title('residuals between '+str(plot_name)+' and Lay94\ny=mx+b, m='+str(coeff[0])+', b='+str(coeff[1])+'\n offset '+str(net_offset))
        plt.savefig(plot_name+'_test_180708.png')
        #plt.show()
        plt.clf()
            
        # return 
        # 1. overlapping Layden94 values
        # 2. FeH values from lit source
        # 3. Residuals between 1. and 2.(see Chadid+ 2017 Figs. 5, 6, 7)
        # 4. coefficients of best-fit line
        # 5. offset in y to bring lit FeH values to match Chadid+ 2017 at FeH=-1.25 (see Chadid+ 2017 Figs. 5, 6)
        # 6. Residuals (from 3.) minus the offset (from 5.)  (see Chadid+ 2017 Fig. 7)
        # 7. The names of the stars (in same order as arrays for 1., 2., 3., 4.)
        
        d = dict()
        d['laydenFeH'] = laydenFeH
        d['inputFeH'] = inputFeH
        d['residuals'] = residuals
        d['coeff'] = coeff
        d['net_offset'] = net_offset
        d['residuals_shifted'] = np.subtract(inputFeH,residuals)
        d['name'] = nameArray
        
        return d

In [ ]:
# now actually find the matches between datasets and apply the offsets

# find matches: Fernley 1996 ## ## WAIT-- FERNLEY 97 INCLUDES THESE
#dict_Fernley_96 = lit_metallicities.find_match_Layden(fernley_feh,layden_feh,'Fernley_96', offset=True)

# find matches: Lambert 1996
dict_Lambert_96 = lit_metallicities.find_match_Layden(lambert_logeps,layden_feh,'Lambert_96', offset=True)

# find matches: Nemec 2013
dict_Nemec_2013  = lit_metallicities.find_match_Layden(nemec_feh,layden_feh,'Nemec_2013', offset=True)

# find matches: Liu 2013
liu_feh2 = liu_feh.groupby(liu_feh['name'], axis=0, as_index=False).mean()
dict_Liu_2013  = lit_metallicities.find_match_Layden(liu_feh2,layden_feh,'Liu_2013', offset=True)

# find matches: Chadid 2017
dict_Chadid_2017  = lit_metallicities.find_match_Layden(chadid_feh,layden_feh,'Chadid_2017', offset=True)

# find matches: Fernley 1997
dict_Fernley_1997  = lit_metallicities.find_match_Layden(fernley97_feh,layden_feh,'Fernley_1997', offset=True)

# find matches: Solano 1997
dict_Solano_1997  = lit_metallicities.find_match_Layden(solano_feh,layden_feh,'Solano_1997', offset=True)

## ## SNEDEN_17 DOES NOT OVERLAP WITH LAYDEN! FIX THIS

# find matches: Wallerstein+ 2010
dict_Wallerstein_2010  = lit_metallicities.find_match_Layden(wallerstein_feh,layden_feh,'Wallerstein_2010', offset=True)

## ## IS THE BELOW NEEDED?
# find matches between Wallerstein and Chadid
# Chadid stars that appear in Wallerstein
#chadid_winnow = lit_metallicities.chadid_feh[chadid_feh['star'].isin(wallerstein_feh['star'])]
# Wallerstein stars that appear in Chadid
#wallerstein_winnow = lit_metallicities.wallerstein_feh[wallerstein_feh['star'].isin(chadid_feh['star'])]

In [ ]:
## APPLY OFFSETS to all datasets and overlap

plt.plot(dict_Lambert_96['laydenFeH'], dict_Lambert_96['residuals_shifted'])
plt.plot(dict_Nemec_2013['laydenFeH'], dict_Nemec_2013['residuals_shifted'])
plt.plot(dict_Liu_2013['laydenFeH'], dict_Liu_2013['residuals_shifted'])
plt.plot(dict_Chadid_2017['laydenFeH'], dict_Chadid_2017['residuals_shifted'])
plt.plot(dict_Fernley_1997['laydenFeH'], dict_Fernley_1997['residuals_shifted'])
plt.plot(dict_Solano_1997['laydenFeH'], dict_Solano_1997['residuals_shifted'])
plt.plot(dict_Wallerstein_2010['laydenFeH'], dict_Wallerstein_2010['residuals_shifted'])
plt.show()

In [ ]:
# merge the metallicity dictionaries

dict_collect = [dict_Lambert_96, dict_Nemec_2013, dict_Liu_2013, dict_Chadid_2017, 
            dict_Fernley_1997, dict_Solano_1997, dict_Wallerstein_2010]
dict_merged = {}
for k in dict_Lambert_96.iterkeys():
    dict_merged[k] = tuple(dict_merged[k] for dict_merged in dict_collect)
    
## ## CAUTION: TEST TO SEE IF THE CONTENT IN THE KEYS IS IN ORDER (I.E., MAKE A PLOT AND SEE IF ITS THE SAME IF DATASETS ARE OVERLAID INDIVIDUALLY)

In [ ]:
# plot merged data and fit linreg line

m_merged,b_merged = polyfit(dict_merged['laydenFeH'], dict_merged['residuals_shifted'], 1)
plt.scatter(dict_merged['laydenFeH'], dict_merged['residuals_shifted'])
plt.plot(dict_merged['laydenFeH'], np.add(np.multiply(dict_merged['laydenFeH'],m_merged),b_merged))
plt.show()

In [ ]:
# CALCULATE FINAL FEH VALUES FOR OUR OWN STARS, AND WRITE OUT 



In [ ]:
# FOR EMCEE, USE CODE rrlyrae_metal_fit_emcee_wrapper.py IN SAME REPOSITORY

In [ ]:
# THEN RUN EMCEE WITH FINAL EWS, ERR_EWS, FEHS, ERR_FEHS

class run_emcee():
    
    ##############################################################################
    # STEP 5: RUN EMCEE ON THE SPACE, GET VALUES FOR a, b, c, d (applicable only to A)
    ##############################################################################
    
    def __init__(self):
        
    def __call__(self):

In [5]:
# read in Ca absorption maps

image, header = fits.getdata('maps_EW(CaNa)_20150318.fits',0,header=True)



In [11]:
print(header)

XTENSION= 'BINTABLE'           / binary table extension                         BITPIX  =                    8 / array data type                                NAXIS   =                    2 / number of array dimensions                     NAXIS1  =                   56 / length of dimension 1                          NAXIS2  =                 3072 / length of dimension 2                          PCOUNT  =                    0 / number of group parameters                     GCOUNT  =                    1 / number of groups                               TFIELDS =                   11 / number of table fields                         TTYPE1  = 'Ca_K_EW '                                                            TFORM1  = 'E       '                                                            TTYPE2  = 'Ca_K_err'                                                            TFORM2  = 'E       '                                                            TTYPE3  = 'Ca_H_EW '                    

In [10]:
[print(i) for i in header]

XTENSION
BITPIX
NAXIS
NAXIS1
NAXIS2
PCOUNT
GCOUNT
TFIELDS
TTYPE1
TFORM1
TTYPE2
TFORM2
TTYPE3
TFORM3
TTYPE4
TFORM4
TTYPE5
TFORM5
TTYPE6
TFORM6
TTYPE7
TFORM7
TTYPE8
TFORM8
TTYPE9
TFORM9
TTYPE10
TFORM10
TTYPE11
TFORM11
NSIDE


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [8]:
print(image['Ca_K_EW '])

[ 0.12263525  0.10066336  0.15257747 ...,  0.          0.          0.        ]
